# Map Reduce

This notebook is performing map reduce in a simplified manner in Python. Distribution of compute to different nodes is not done here; the purpose rather is to explore how to implement a map or reduce function, assuming that the functionality is provided akin to the libraries mentioned in [Dean and Ghemawat](https://static.googleusercontent.com/media/research.google.com/en//archive/mapreduce-osdi04.pdf).


This notebook comprises a section defining identity mappers and reducers, along with a `run` method which you may change if necessary. An intermediate sort function is also provided.

Implement the `mapper` and `reducer` in the Term Vectors section, and use the run cell as provided.


In [ ]:
from itertools import groupby
from operator import itemgetter
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
%config Completer.use_jedi = False


[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Empty MAPPER
def mapper(key, value):
    """
    Our user defined mapper function .
    : param key :
    : param value :
    """
    yield (key, value)

In [ ]:
# Empty REDUCER

def reducer(key , list_value):
    """
    User defined reducer.
    : param key :
    : param list_value :
    """
    yield (key, list_value)

In [ ]:


def cleaner(line):
    # lowercase all words and get alphabetical char only and keeping
    # apostrophe for time being
    words = re.findall(r'[a-z\']+' , line.lower())
    for word in words :
        # we will omit apostrophe ' s assuming users won't type them in a search
        word = word.replace("'" , '')
        if not (word is '' or word in stopwords.words('english')):
            yield word



def intermediate_sort(data):
    """
    collect by key
    """
    data = sorted ( data )
    return [(k, list(tuple(zip(*g))[1])) for k, g  in groupby(data , itemgetter(0))]



def run(sources_dict):
    """
    Since we are focusing on the mapper and reducer functions here we have to
    provide the boiler plate code that a MapReduce library typically would . This
    function does that in a simple way (we ignore distributing it for now).
    : param sources_dict : dictionary where (key,fqfilename), for example ('doc_id','/home/fileX')
    """
    map_result =[]
    reduce_result =[]
    # open the files and apply map to each of them ( could be done in parallel ,
    # but we prefer to keep it simple ) .
    for k , v in sources_dict.items():
        # do map per source
        # this could happen in its own process / worker typically
        f = open(v, 'r')
        map_result += list(mapper(k, f.read()))
        f.close()
#         ::alt
#          with open(v, 'r') as f:
#             for line in f.readlines():
#                 map_result += list(mapper(k, line))
    # this would be written to disk in the original paradigm ,
    # but we keep it in memory for ease of use
    intermediate_result = intermediate_sort(map_result)
    # now that the data has been ' collected ' and grouped by key it can be handed
    # to the reducers . They would run over partitions or chunks usually , but we
    # will just iterate through the keys we have and call them
    for elem in intermediate_result:
        reduce_result.append(list(reducer(elem [0], elem [1])))
    return map_result, intermediate_result, reduce_result

In [ ]:
# EXAMPLE
!mkdir -p input/
!echo -e 'D1 : the cat sat on the mat' > input/d1.txt
!echo -e 'D2 : the dog sat on the log' > input/d2.txt

_, _, res = run({'D1': 'input/d1.txt' , 'D2': 'input/d2.txt'})

res

[[('D1', ['D1 : the cat sat on the mat\n'])],
 [('D2', ['D2 : the dog sat on the log\n'])]]

# Term Vector

The paper states:

> Term-Vector per Host: A term vector summarizes the
most important words that occur in a document or a set
of documents as a list of 〈word, frequency〉 pairs. The
map function emits a 〈hostname, term vector〉
pair for each input document (where the hostname is
extracted from the URL of the document). The re-
duce function is passed all per-document term vectors
for a given host. It adds these term vectors together,
throwing away infrequent terms, and then emits a final
〈hostname, term vector〉 pair.

As for

> throwing away infrequent terms

Write your code in such a way that only terms occurring at least twice are retained.

Hint:
  * Consider how they use the word 'frequency' elsewhere in the paper.


In [ ]:
# your mapper
def mapper(key, value):
    yield (key, value)

def reducer(key , list_value):
    yield (key, list_value)

In [26]:
x, y, res = run({'www.somesite.com/page/1': 'page1.txt', 'www.somesite.com/page/2': 'page2.txt'})

FileNotFoundError: [Errno 2] No such file or directory: 'page1.txt'

In [25]:
from collections import Counter

def mapper(key, value):
    """
    Our user defined mapper function.
    :param key: document URL
    :param value: document content
    """
    hostname = key.split('/')[2]  # Extract hostname from URL
    term_vector = Counter(cleaner(value))
    yield (hostname, dict(term_vector))

def reducer(key, list_value):
     """
     User defined reducer.
     :param key: hostname
     :param list_value: list of term vectors for the host
      """
     combined_vector = Counter()

                                                                # Check if list_value is already a list of term vectors
if isinstance(list_value, list) and all(isinstance(item, dict) for item in list_value):
                                                               for term_vector in list_value:
                                                                   combined_vector.update(term_vector)
                                                               else:
                                                                                                    # If it's a single term vector, just update with it
                                                                  combined_vector.update(list_value)
from collections import Counter

def mapper(key, value):
    """
    Our user defined mapper function.
    :param key: document URL
    :param value: document content
    """
    hostname = key.split('/')[2]  # Extract hostname from URL
    term_vector = Counter(cleaner(value))
    yield (hostname, dict(term_vector))

def reducer(key, list_value):
     """
     User defined reducer.
     :param key: hostname
     :param list_value: list of term vectors for the host
      """
     combined_vector = Counter()

     # Check if list_value is already a list of term vectors
     if isinstance(list_value, list) and all(isinstance(item, dict) for item in list_value):
         for term_vector in list_value:
             combined_vector.update(term_vector)
     else:
         # If it's a single term vector, just update with it
         combined_vector.update(list_value)

     filtered_vector = {word: count for word, count in combined_vector.items() if count >= 2}
     yield (key, filtered_vector)


NameError: name 'list_value' is not defined